## Import Packages

In [15]:
import pandas as pd
import os
from google.cloud import vision
from google.cloud.vision_v1 import types

## Task A

In [16]:
# Read in data
raw = pd.read_csv("NatGeo Scrape 11-27.csv")

In [17]:
raw.head()

,postUrl,description,commentCount,likeCount,type,imgUrl
0,https://www.instagram.com/p/ClevlmAA0wj/,Photo by @mattiasklumofficial | A painted dog ...,24,870,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...
1,https://www.instagram.com/p/CleNQPAgxgR/,Photo by @acacia.johnson | Inuit elder Peugatu...,296,68145,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...
2,https://www.instagram.com/p/Cld8FxSMe7U/,Photo by @gerdludwig | Visible from the 75th f...,365,48060,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...
3,https://www.instagram.com/p/ClbodXJA72X/,Photo by @paoloverzone | At the Egyptian Museu...,376,46835,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...
4,https://www.instagram.com/p/ClbXSkWgZHw/,Photo by Orsolya Haarberg @fjellheimengalleri ...,419,73279,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...


In [18]:
# Normalize function
def normalize(df, column, rowVal):
    """ Normalize values within a column"""
    sumCount = df[column].max()
    return float(rowVal) / float(sumCount)

def binary(val):
    """ Determine engagement score to be either 0 or 1"""
    median = raw["EngagementScore"].median()
    if val >= median:
        return 1
    elif val < median:
        return 0

In [19]:
# Normalizing # of likes and # of comments
raw["NormLikes"] = raw["likeCount"].apply(lambda x: normalize(raw, "likeCount", x))
raw["NormComments"] = raw["commentCount"].apply(lambda x: normalize(raw, "commentCount", x))

# Create engagement score
raw["EngagementScore"] = raw["NormLikes"].apply(lambda x: x * .4) + raw["NormComments"].apply(lambda x: x * .6)

# Create binary engagement score
raw["BinEngagementScore"] = raw["EngagementScore"].apply(lambda x: binary(x))

In [20]:
raw.head()

,postUrl,description,commentCount,likeCount,type,imgUrl,NormLikes,NormComments,EngagementScore,BinEngagementScore
0,https://www.instagram.com/p/ClevlmAA0wj/,Photo by @mattiasklumofficial | A painted dog ...,24,870,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.000908,0.000500,0.000664,0
1,https://www.instagram.com/p/CleNQPAgxgR/,Photo by @acacia.johnson | Inuit elder Peugatu...,296,68145,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.071139,0.006172,0.032159,0
2,https://www.instagram.com/p/Cld8FxSMe7U/,Photo by @gerdludwig | Visible from the 75th f...,365,48060,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.050172,0.007611,0.024635,0
3,https://www.instagram.com/p/ClbodXJA72X/,Photo by @paoloverzone | At the Egyptian Museu...,376,46835,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.048893,0.007840,0.024261,0
4,https://www.instagram.com/p/ClbXSkWgZHw/,Photo by Orsolya Haarberg @fjellheimengalleri ...,419,73279,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.076499,0.008737,0.035842,0


## Google Vision

In [21]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'summer-optics-369919-d156981281b2.json'
client = vision.ImageAnnotatorClient()

In [22]:
def googleVision(img):
    """ Getting image labels """
    image = types.Image()
    image.source.image_uri = img

    response_label = client.label_detection(image=image)

    imgLabel = [label.description for label in response_label.label_annotations]

    return imgLabel

In [24]:
raw["Image Labels"] = raw["imgUrl"].apply(lambda x: googleVision(x))

In [25]:
raw.head()

,postUrl,description,commentCount,likeCount,type,imgUrl,NormLikes,NormComments,EngagementScore,BinEngagementScore,Image Labels
0,https://www.instagram.com/p/ClevlmAA0wj/,Photo by @mattiasklumofficial | A painted dog ...,24,870,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.000908,0.000500,0.000664,0,"[Eye, Carnivore, Plant, Whiskers, Grass, Fawn,..."
1,https://www.instagram.com/p/CleNQPAgxgR/,Photo by @acacia.johnson | Inuit elder Peugatu...,296,68145,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.071139,0.006172,0.032159,0,"[Sky, Ice cap, Horizon, Landscape, Electric bl..."
2,https://www.instagram.com/p/Cld8FxSMe7U/,Photo by @gerdludwig | Visible from the 75th f...,365,48060,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.050172,0.007611,0.024635,0,"[Cloud, Sky, Atmosphere, Building, World, Dusk..."
3,https://www.instagram.com/p/ClbodXJA72X/,Photo by @paoloverzone | At the Egyptian Museu...,376,46835,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.048893,0.007840,0.024261,0,"[Table, Shelf, Wood, Cabinetry, Drawer, Displa..."
4,https://www.instagram.com/p/ClbXSkWgZHw/,Photo by Orsolya Haarberg @fjellheimengalleri ...,419,73279,Photo,https://scontent-lga3-2.cdninstagram.com/v/t51...,0.076499,0.008737,0.035842,0,"[Sky, Snow, Ecoregion, Slope, Natural landscap..."


## Task B

## Task C

## Task D